
# LLM for outputting a user-friendly message

Use the trained YAMNet+context model to produce predictions and ask a Groq LLM to turn them into clear guidance for end users.



### Loading model, data + encoder

Mount Drive and load the saved classifier, label encoder, and test arrays needed for inference.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from tensorflow.keras.models import load_model
import numpy as np
import joblib
import json

# Load the trained multimodal classifier
model = load_model('/content/drive/MyDrive/project_samsung_IA/yamnet_model.keras')

# Load label encoder to turn indices into human-readable classes
le = joblib.load('/content/drive/MyDrive/project_samsung_IA/label_encoder.plk')

# Cached test splits to reuse during inference demos
X_audio_test = np.load('/content/drive/MyDrive/project_samsung_IA/X_audio_test.npy')
X_context_test = np.load('/content/drive/MyDrive/project_samsung_IA/X_context_test.npy')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



### Install Groq client (Colab runtime)

Install the SDK in the runtime so we can call the hosted LLM.


In [ ]:
!pip install groq  # One-time install per runtime


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 3.6 MB/s eta 0:00:00



### Configure Groq client

Initialize the client with your API key (ideally set as an environment variable instead of hard-coding it).


In [ ]:
from groq import Groq

# Replace the string with an environment variable for safer use
client = Groq(api_key="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")



### Prompt builder for structured explanations

Build a JSON-only prompt that keeps each predicted issue independent and outputs user-friendly advice.


In [ ]:
def generate_message_json(top1_problem, top1_prob, top2_problem=None, top2_prob=None):
    """Create a JSON-only prompt for the Groq LLM and parse its response."""

    # -------------------------------
    # Dynamic input section
    # -------------------------------
    if top2_problem is None:
        prediction_section = f"""
Top-1 prediction:
- Problem: {top1_problem}
- Probability: {top1_prob:.3f}
"""
    else:
        prediction_section = f"""
Top-2 predictions:
1. {top1_problem} — {top1_prob:.3f}
2. {top2_problem} — {top2_prob:.3f}
"""

    # -------------------------------
    # FULL PROMPT (forces JSON output + strict independence rules)
    # -------------------------------
    full_prompt = f"""
You are an automotive diagnostic assistant.

Below is the prediction data:

{prediction_section}

TASK:
Using the predictions, generate a structured JSON object following these rules:

1. If the top-1 probability is greater than 70%, include ONLY the top-1 problem.
2. Otherwise, include BOTH top-1 and top-2 problems.

For each included problem, return fields with rich and helpful content:

- problem:
  The exact problem label (string).

- probability:
  Probability as a float between 0 and 1.

- severity:
  A classification ("low", "moderate", "high") plus a short explanation.

- description:
  A detailed, friendly explanation of:
    - what this problem means
    - why it typically happens (generic causes only)
    - how it affects the vehicle
  DO NOT mention or imply any other problem label.

- recommended_action:
  Step-by-step instructions on what the user should do.
  Indicate urgency.

- what_to_tell_mechanic:
  A short, direct sentence the user can say.

- advice:
  Practical tips to prevent or monitor the issue.

---------------------------------------
STRICT RULES (VERY IMPORTANT):
---------------------------------------

1. Each problem label is independent. Never mix causes, explanations, or symptoms between labels.
   Forbidden examples:
   - "car_knocking happens because of low_oil"
   - "dead_battery is caused by bad_ignition"
   - "worn_out_brakes lead to normal_brakes"
   - "power_steering problems are caused by serpentine_belt issues"

2. You MUST NOT describe one predicted problem as being caused by another problem label.

3. ONLY describe the selected label based on generic automotive knowledge, not the meaning of any other label.

4. If a real-world cause overlaps with another label, DO NOT USE IT.
   Instead, provide a general explanation that does NOT reference or imply any other label.

5. Forbidden cause categories include (but are not limited to):
   - oil level, oil pressure, lubrication problems (related to low_oil)
   - battery charge, alternator failure (related to dead_battery)
   - ignition failure, spark issues (related to bad_ignition)
   - belt tension, pulley problems (related to serpentine_belt)
   - brake wear, pad thickness (related to worn_out_brakes)
   - hydraulic steering loss (related to power_steering)
   - normal behavior / normal operation (related to normal_* labels)

6. If describing a cause might violate the rules, provide a SAFE GENERIC cause such as:
   "general component wear", "sensor irregularities", "normal variation", etc.

7. Before producing the JSON, internally verify:
   “Did I accidentally reference or imply another label or its causes?”
   If YES → Rewrite the explanation so NO cross-label interference exists.

---------------------------------------
OUTPUT FORMAT (MANDATORY):
---------------------------------------

Return ONLY a valid JSON object using EXACTLY this structure:

{
  "issues": [
    {
      "problem": "...",
      "probability": 0.95,
      "severity": "...",
      "description": "...",
      "recommended_action": "...",
      "what_to_tell_mechanic": "...",
      "advice": "..."
    }
  ]
}

- No markdown.
- No extra commentary.
- No text outside the JSON.
- Must be valid JSON.
"""

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": full_prompt}],
        temperature=0.2,
        max_tokens=500
    )

    # Parse JSON safely
    raw_output = response.choices[0].message.content.strip()

    try:
        decoded = json.loads(raw_output)
    except json.JSONDecodeError:
        # Attempt extraction fallback
        try:
            json_start = raw_output.index("{")
            json_end = raw_output.rindex("}") + 1
            decoded = json.loads(raw_output[json_start:json_end])
        except Exception:
            decoded = {"error": "LLM returned invalid JSON", "raw": raw_output}

    return decoded



### Run model inference and request explanation

Call the trained model to get class probabilities, then format a prompt for the LLM based on confidence levels.


In [ ]:
def full_diagnosis(i):
    """Run inference on the test split, decide whether to ask for top-1 or top-2, and fetch the JSON explanation."""

    # Prepare model inputs
    emb = X_audio_test[i].reshape(1, -1)
    ctx = np.array([[int(X_context_test[i])]], dtype=np.int32)

    # Predictions
    probs = model.predict([emb, ctx], verbose=0)[0]

    # Top 3 ordered
    top_idx = probs.argsort()[-3:][::-1]

    top1_class = le.classes_[top_idx[0]]
    top1_prob = float(probs[top_idx[0]])

    # Case 1 → Top-1 only
    if top1_prob >= 0.70:
        explanation_json = generate_message_json(
            top1_problem=top1_class,
            top1_prob=top1_prob
        )
        return {
            "mode": "single",
            "prediction": top1_class,
            "confidence": top1_prob,
            "explanation_json": explanation_json
        }

    # Case 2 → Top-2
    top2_class = le.classes_[top_idx[1]]
    top2_prob = float(probs[top_idx[1]])

    explanation_json = generate_message_json(
        top1_problem=top1_class,
        top1_prob=top1_prob,
        top2_problem=top2_class,
        top2_prob=top2_prob
    )

    return {
        "mode": "top2",
        "predictions": [
            (top1_class, top1_prob),
            (top2_class, top2_prob)
        ],
        "explanation_json": explanation_json
    }



### Example calls

Sample invocations on different test indices. Adjust the index to explore other audio clips.


In [ ]:
# High-confidence example
result = full_diagnosis(150)
result


{'mode': 'single',
 'prediction': np.str_('car_knocking'),
 'confidence': 0.9947447776794434,
 'explanation_json': {'issues': [{'problem': 'car_knocking',
    'probability': 0.995,
    'severity': 'high',
    'description': "Car knocking is a loud, metallic knocking or tapping sound coming from the engine. It typically happens due to general component wear, which can cause the engine's moving parts to rub against each other. This can lead to engine damage and decreased performance. The knocking sound can be heard when the engine is running, especially when accelerating or decelerating.",
    'recommended_action': 'Urgent: Have your car checked by a mechanic as soon as possible. They will inspect the engine and perform any necessary repairs to prevent further damage.',
    'what_to_tell_mechanic': "The car is making a loud knocking sound and I'm worried it might be engine-related.",
    'advice': "Regular oil changes and tune-ups can help prevent general component wear. Monitor your eng

In [ ]:
# Lower-confidence example to trigger top-2
result = full_diagnosis(10)
result


{'mode': 'single',
 'prediction': np.str_('normal_engine_idle'),
 'confidence': 0.9999988079071045,
 'explanation_json': {'issues': [{'problem': 'normal_engine_idle',
    'probability': 1.0,
    'severity': 'low',
    'description': "The engine is idling normally, which means it's running at a steady speed without any unusual vibrations or noises. This typically happens when the engine is in good condition and the idle air control system is functioning properly. It can be affected by general component wear or sensor irregularities.",
    'recommended_action': 'No action is required. However, if you notice any unusual engine behavior, consult a mechanic.',
    'what_to_tell_mechanic': 'The engine is idling normally.',
    'advice': 'Regularly check the engine oil level and condition to prevent general component wear.'}]}}

In [ ]:
import json  # Handy for pretty-printing results during exploration
# Example: print(json.dumps(full_diagnosis(5), indent=2))


In [ ]:
result = full_diagnosis(5)
result


{'mode': 'single',
 'prediction': np.str_('car_knocking'),
 'confidence': 0.9812825918197632,
 'explanation_json': {'issues': [{'problem': 'car_knocking',
    'probability': 0.981,
    'severity': 'high',
    'description': "Car knocking is a loud clunking or tapping noise coming from the engine. It typically happens due to general component wear, which can cause the engine's internal components to collide and produce the noise. This can affect the engine's performance and longevity. If left unchecked, it can lead to engine damage.",
    'recommended_action': 'Urgent: Schedule an engine inspection with a mechanic as soon as possible. They will diagnose the issue and recommend the necessary repairs.',
    'what_to_tell_mechanic': 'The engine is making a loud knocking noise.',
    'advice': "Regular oil changes and tune-ups can help prevent general component wear. Monitor the engine's performance and report any unusual noises to a mechanic promptly."}]}}


The `explanation_json` returned by `full_diagnosis` can be fed directly into a UI or voice assistant to give drivers friendly, actionable guidance.
